# ** Weather Data Analysis **

## 1. Historische Wetterdaten müssen aus einer CSV-Datei geladen werden

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns


#data lesen
df = pd.read_parquet("daily_weather.parquet")

# daten anzeigen
print("daten (erste 5 Zeilen):")
print(df.head())

daten (erste 5 Zeilen):
  station_id city_name       date  season  avg_temp_c  min_temp_c  max_temp_c  \
0      41515  Asadabad 1957-07-01  Summer        27.0        21.1        35.6   
1      41515  Asadabad 1957-07-02  Summer        22.8        18.9        32.2   
2      41515  Asadabad 1957-07-03  Summer        24.3        16.7        35.6   
3      41515  Asadabad 1957-07-04  Summer        26.6        16.1        37.8   
4      41515  Asadabad 1957-07-05  Summer        30.8        20.0        41.7   

   precipitation_mm  snow_depth_mm  avg_wind_dir_deg  avg_wind_speed_kmh  \
0               0.0            NaN               NaN                 NaN   
1               0.0            NaN               NaN                 NaN   
2               1.0            NaN               NaN                 NaN   
3               4.1            NaN               NaN                 NaN   
4               0.0            NaN               NaN                 NaN   

   peak_wind_gust_kmh  avg_sea_l

## 2. Alle Datumswerte sind korrekt zu interpretieren und zu standardisieren

In [42]:
#  Datum standardisieren 
df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.strftime('%Y-%m-%d')

## 3.Fehlende oder fehlerhafte Wetterdaten sollen erkannt und bereinigt werden. 

In [43]:
# Fehlerhafte Werte erkennen
# Annahmen für Fehler:
# - Temperaturen: min_temp_c < -50 oder max_temp_c > 50 sind unrealistisch
# - avg_temp_c sollte zwischen min_temp_c und max_temp_c liegen
# - Niederschlag: precipitation_mm < 0 ist ungültig
temp_errors = df[(df['min_temp_c'] < -50) | (df['max_temp_c'] > 50) | 
                 (df['avg_temp_c'] < -50) | (df['avg_temp_c'] > 50) |
                 (df['avg_temp_c'] < df['min_temp_c']) | (df['avg_temp_c'] > df['max_temp_c'])]
precip_errors = df[df['precipitation_mm'] < 0]

print("\nFehlerhafte Temperaturwerte:")
print(temp_errors)
print("\nFehlerhafte Niederschlagswerte:")
print(precip_errors)


Fehlerhafte Temperaturwerte:
      station_id city_name        date  season  avg_temp_c  min_temp_c  \
5202       38954  Fayzabad  1914-02-28  Winter        -3.3        -3.2   
6228       38954  Fayzabad  1916-12-20  Winter        -5.6        -4.5   
6229       38954  Fayzabad  1916-12-21  Winter        -6.3        -2.4   
6230       38954  Fayzabad  1916-12-22  Winter        -5.9        -1.9   
6231       38954  Fayzabad  1916-12-23  Winter        -5.8        -2.1   
...          ...       ...         ...     ...         ...         ...   
9693       67975  Masvingo  1979-12-04  Summer        19.5        20.6   
10330      67975  Masvingo  1981-09-01  Spring        13.4        12.8   
10548      67975  Masvingo  1982-04-07  Autumn        29.4        13.4   
11114      67975  Masvingo  1983-10-25  Spring        17.4        18.1   
12709      67975  Masvingo  1988-03-07  Autumn        19.7        19.9   

       max_temp_c  precipitation_mm  snow_depth_mm  avg_wind_dir_deg  \
5202     

In [ ]:
# Daten bereinigen
# Fehlerhafte Werte zu NaN setzen
df['avg_temp_c'] = np.where((df['avg_temp_c'] < -50) | (df['avg_temp_c'] > 50) | 
                            (df['avg_temp_c'] < df['min_temp_c']) | (df['avg_temp_c'] > df['max_temp_c']), 
                            np.nan, df['avg_temp_c'])
df['min_temp_c'] = np.where(df['min_temp_c'] < -50, np.nan, df['min_temp_c'])
df['max_temp_c'] = np.where(df['max_temp_c'] > 50, np.nan, df['max_temp_c'])
df['precipitation_mm'] = np.where(df['precipitation_mm'] < 0, np.nan, df['precipitation_mm'])

# Fehlende Werte interpolieren
df['avg_temp_c'] = df['avg_temp_c'].interpolate(method='linear')
df['min_temp_c'] = df['min_temp_c'].interpolate(method='linear')
df['max_temp_c'] = df['max_temp_c'].interpolate(method='linear')
df['precipitation_mm'] = df['precipitation_mm'].interpolate(method='linear')

In [ ]:
# Bereinigte Daten anzeigen
print("\nBereinigte Daten :")
print(df.head())



Bereinigte Daten :
  station_id city_name        date  season  avg_temp_c  min_temp_c  \
0      41515  Asadabad  1957-07-01  Summer        27.0        21.1   
1      41515  Asadabad  1957-07-02  Summer        22.8        18.9   
2      41515  Asadabad  1957-07-03  Summer        24.3        16.7   
3      41515  Asadabad  1957-07-04  Summer        26.6        16.1   
4      41515  Asadabad  1957-07-05  Summer        30.8        20.0   

   max_temp_c  precipitation_mm  snow_depth_mm  avg_wind_dir_deg  \
0        35.6               0.0            NaN               NaN   
1        32.2               0.0            NaN               NaN   
2        35.6               1.0            NaN               NaN   
3        37.8               4.1            NaN               NaN   
4        41.7               0.0            NaN               NaN   

   avg_wind_speed_kmh  peak_wind_gust_kmh  avg_sea_level_pres_hpa  \
0                 NaN                 NaN                     NaN   
1           

## 4. Temperaturen ggf. in Celsius (oder andere Einheiten) umrechnen.

In [ ]:
# Temperaturen ggf. in eine andere Einheit umrechnen
# Annahme: Daten sind aktuell in Celsius, Ziel ist z. B. Fahrenheit
aktuelle_einheit = "celsius"  
ziel_einheit = "fahrenheit"  

# Umrechnungsfunktionen definieren
def celsius_zu_fahrenheit(c):
    return (c * 9/5) + 32

def fahrenheit_zu_celsius(f):
    return (f - 32) * 5/9

In [ ]:
# Umrechnung durchführen
if aktuelle_einheit.lower() != ziel_einheit.lower():
    if ziel_einheit.lower() == "fahrenheit" and aktuelle_einheit.lower() == "celsius":
        df['avg_temp_c'] = df['avg_temp_c'].apply(celsius_zu_fahrenheit)
        df['min_temp_c'] = df['min_temp_c'].apply(celsius_zu_fahrenheit)
        df['max_temp_c'] = df['max_temp_c'].apply(celsius_zu_fahrenheit)
        # Spalten umbenennen
        #df = df.rename(columns={
         #   'avg_temp_c': 'avg_temp_f',
         #   'min_temp_c': 'min_temp_f',
         #   'max_temp_c': 'max_temp_f'
        #})
        print(f"\nTemperaturen von {aktuelle_einheit} nach {ziel_einheit} umgerechnet.")
    elif ziel_einheit.lower() == "celsius" and aktuelle_einheit.lower() == "fahrenheit":
        df['avg_temp_c'] = df['avg_temp_c'].apply(fahrenheit_zu_celsius)
        df['min_temp_c'] = df['min_temp_c'].apply(fahrenheit_zu_celsius)
        df['max_temp_c'] = df['max_temp_c'].apply(fahrenheit_zu_celsius)
        print(f"\nTemperaturen von {aktuelle_einheit} nach {ziel_einheit} umgerechnet.")
    else:
        print("\nFehler: Ungültige Einheitenkombination.")
else:
    print(f"\nKeine Umrechnung nötig, Daten sind bereits in {aktuelle_einheit}.")


Temperaturen von celsius nach fahrenheit umgerechnet.


In [ ]:
print("\nBereinigte und umgerechnete Daten :")
print(df.head())


Bereinigte und umgerechnete Daten :
  station_id city_name        date  season  avg_temp_c  min_temp_c  \
0      41515  Asadabad  1957-07-01  Summer       80.60       69.98   
1      41515  Asadabad  1957-07-02  Summer       73.04       66.02   
2      41515  Asadabad  1957-07-03  Summer       75.74       62.06   
3      41515  Asadabad  1957-07-04  Summer       79.88       60.98   
4      41515  Asadabad  1957-07-05  Summer       87.44       68.00   

   max_temp_c  precipitation_mm  snow_depth_mm  avg_wind_dir_deg  \
0       96.08               0.0            NaN               NaN   
1       89.96               0.0            NaN               NaN   
2       96.08               1.0            NaN               NaN   
3      100.04               4.1            NaN               NaN   
4      107.06               0.0            NaN               NaN   

   avg_wind_speed_kmh  peak_wind_gust_kmh  avg_sea_level_pres_hpa  \
0                 NaN                 NaN                     Na

## 5. Berechnung der täglichen Durchschnittstemperaturen.

In [ ]:
#Berechnung der täglichen Durchschnittstemperaturen
# Neue Spalte mit Durchschnitt aus min_temp_f und max_temp_f berechnen
df['tag_avg_temp_c'] = (df['min_temp_c'] + df['max_temp_c']) / 2

print(df.head())

  station_id city_name        date  season  avg_temp_c  min_temp_c  \
0      41515  Asadabad  1957-07-01  Summer       80.60       69.98   
1      41515  Asadabad  1957-07-02  Summer       73.04       66.02   
2      41515  Asadabad  1957-07-03  Summer       75.74       62.06   
3      41515  Asadabad  1957-07-04  Summer       79.88       60.98   
4      41515  Asadabad  1957-07-05  Summer       87.44       68.00   

   max_temp_c  precipitation_mm  snow_depth_mm  avg_wind_dir_deg  \
0       96.08               0.0            NaN               NaN   
1       89.96               0.0            NaN               NaN   
2       96.08               1.0            NaN               NaN   
3      100.04               4.1            NaN               NaN   
4      107.06               0.0            NaN               NaN   

   avg_wind_speed_kmh  peak_wind_gust_kmh  avg_sea_level_pres_hpa  \
0                 NaN                 NaN                     NaN   
1                 NaN           

## 6. Identikation von Maximal- und Minimaltemperaturen.


In [ ]:
# Identifikation von Maximal- und Minimaltemperaturen
global_max_temp = df['max_temp_c'].max()
global_min_temp = df['min_temp_c'].min()

print ("max_temp :", global_max_temp , ziel_einheit )
print ("min_temp : ", global_min_temp ,   ziel_einheit)

max_temp : 122.0 fahrenheit
min_temp :  -58.0 fahrenheit


## 7. Berechnung von Mittelwert, Median und Standardabweichung für verschiedene Parameter.

In [ ]:
# Durchschnitt berechnen
mean_temp = df['tag_avg_temp_c'].mean()
mean_precipitation = df['precipitation_mm'].mean()

# Median berechnen
median_temp = df['tag_avg_temp_c'].median()
median_precipitation = df['precipitation_mm'].median()

# Standardabweichung berechnen
std_temp = df['tag_avg_temp_c'].std()
std_precipitation = df['precipitation_mm'].std()

# Ergebnisse anzeigen
print(f"\nDurchschnittswerte:")
print(f"Durchschnittstemperatur pro Tag: {mean_temp} {ziel_einheit}")
print(f"Durchschnittlicher Niederschlag: {mean_precipitation} mm")

print(f"\nMedianwerte:")
print(f"Median der täglichen Temperatur: {median_temp} {ziel_einheit}")
print(f"Median des Niederschlags: {median_precipitation} mm")

print(f"\nStandardabweichung:")
print(f"Standardabweichung der täglichen Temperatur: {std_temp} {ziel_einheit}")
print(f"Standardabweichung des Niederschlags: {std_precipitation} mm")



Durchschnittswerte:
Durchschnittstemperatur pro Tag: 59.934360103609094 fahrenheit
Durchschnittlicher Niederschlag: 3.4651448360589763 mm

Medianwerte:
Median der täglichen Temperatur: 63.23 fahrenheit
Median des Niederschlags: 0.0 mm

Standardabweichung:
Standardabweichung der täglichen Temperatur: 20.95714387359994 fahrenheit
Standardabweichung des Niederschlags: 12.072316722883393 mm


In [ ]:

mean_temp = df['tag_avg_temp_c'].mean()
median_temp = df['tag_avg_temp_c'].median()
std_temp = df['tag_avg_temp_c'].std()


summary_stats = pd.DataFrame({
    'Statistic': ['Durchschnitt', 'Median', 'Standardabweichung'],
    'Wert': [mean_temp, median_temp, std_temp]
})

fig = px.bar(summary_stats, x='Statistic', y='Wert', 
             title='Vergleich von Durchschnitt, Median und Standardabweichung',
             labels={'Wert': f'Durchschnittstemperatur ({ziel_einheit})', 'Statistic': 'Statistik'},
             color='Statistic', color_discrete_map={'Durchschnitt': 'blue', 'Median': 'green', 'Standardabweichung': 'red'})


fig.show()


8.  Analyse und Visualisierung saisonaler Muster (z. B. Sommer vs. Winter).


In [ ]:

# 📌 1. Datum in das richtige Format umwandeln
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# 📌 2. Saisonale Klassifikation (Frühling, Sommer, Herbst, Winter)
df['month'] = df['date'].dt.month
df['season'] = np.where(df['month'].isin([12, 1, 2]), 'Winter', 
                        np.where(df['month'].isin([3, 4, 5]), 'Frühling', 
                                 np.where(df['month'].isin([6, 7, 8]), 'Sommer', 
                                          np.where(df['month'].isin([9, 10, 11]), 'Herbst', 'Übergang'))))

# 📌 3. Daten nach Saison gruppieren und Mittelwerte berechnen
df_seasonal = df.groupby('season').agg({
    'avg_temp_c': 'mean',
    'min_temp_c': 'mean',
    'max_temp_c': 'mean',
    'precipitation_mm': 'mean'
}).reset_index()

# 📌 4. Temperatur-Visualisierung nach Saison
fig_temp = go.Figure()

fig_temp.add_trace(go.Bar(x=df_seasonal['season'], y=df_seasonal['avg_temp_c'], 
                          name='Durchschnittstemperatur', marker_color='blue'))
fig_temp.add_trace(go.Bar(x=df_seasonal['season'], y=df_seasonal['max_temp_c'], 
                          name='Maximale Temperatur', marker_color='red'))
fig_temp.add_trace(go.Bar(x=df_seasonal['season'], y=df_seasonal['min_temp_c'], 
                          name='Minimale Temperatur', marker_color='green'))

fig_temp.update_layout(title='Saisonale Temperaturunterschiede (Frühling, Sommer, Herbst, Winter)',
                       xaxis_title='Saison',
                       yaxis_title='Temperatur (°C)',
                       legend_title='Temperaturwerte',
                       barmode='group')

# 📌 5. Niederschlags-Visualisierung nach Saison
fig_rain = go.Figure()

fig_rain.add_trace(go.Bar(x=df_seasonal['season'], y=df_seasonal['precipitation_mm'], 
                          name='Durchschnittlicher Niederschlag', marker_color='purple'))

fig_rain.update_layout(title='Saisonale Niederschlagsunterschiede (Frühling, Sommer, Herbst, Winter)',
                       xaxis_title='Saison',
                       yaxis_title='Niederschlag (mm)',
                       legend_title='Niederschlagswerte')

# 📌 6. Diagramme anzeigen
fig_temp.show()
fig_rain.show()


## 9. Erstellung interaktiver Visualisierungen (z. B. Liniendiagramme zur Temperaturentwicklung).

In [ ]:

# 📌 1. Daten laden (hier als Platzhalter, da die Datei nicht verfügbar ist)
# df = pd.read_parquet("daily_weather.parquet")  

# 📌 2. Datum in das richtige Format umwandeln
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# 📌 3. Fehlerhafte Werte identifizieren und bereinigen
df.loc[(df['min_temp_c'] < -50) | (df['max_temp_c'] > 50), ['min_temp_c', 'max_temp_c']] = np.nan
df.loc[(df['avg_temp_c'] < df['min_temp_c']) | (df['avg_temp_c'] > df['max_temp_c']), 'avg_temp_c'] = np.nan
df.loc[df['precipitation_mm'] < 0, 'precipitation_mm'] = np.nan

# 📌 4. Fehlende Werte interpolieren
df['avg_temp_c'] = df['avg_temp_c'].interpolate()
df['min_temp_c'] = df['min_temp_c'].interpolate()
df['max_temp_c'] = df['max_temp_c'].interpolate()
df['precipitation_mm'] = df['precipitation_mm'].interpolate()

# 📌 5. 20-jährige Gruppen erstellen
df['year'] = df['date'].dt.year
df['year_group'] = (df['year'] // 20) * 20  # Gruppierung in 20-Jahres-Schritten

# 📌 6. Durchschnittswerte für jede Gruppe berechnen
df_grouped = df.groupby('year_group').agg({
    'avg_temp_c': 'mean',
    'min_temp_c': 'mean',
    'max_temp_c': 'mean',
    'precipitation_mm': 'mean'
}).reset_index()

# 📌 7. Interaktives Diagramm für die Temperatur erstellen
fig_temp = go.Figure()

fig_temp.add_trace(go.Scatter(x=df_grouped['year_group'], y=df_grouped['avg_temp_c'], 
                              mode='lines+markers', name='Durchschnittstemperatur', line=dict(color='blue')))
fig_temp.add_trace(go.Scatter(x=df_grouped['year_group'], y=df_grouped['max_temp_c'], 
                              mode='lines+markers', name='Maximale Temperatur', line=dict(color='red')))
fig_temp.add_trace(go.Scatter(x=df_grouped['year_group'], y=df_grouped['min_temp_c'], 
                              mode='lines+markers', name='Minimale Temperatur', line=dict(color='green')))

fig_temp.update_layout(title='Temperaturentwicklung alle 20 Jahre',
                       xaxis_title='Jahr (20-Jahres-Gruppen)',
                       yaxis_title='Temperatur (°C)',
                       legend_title='Temperaturwerte',
                       hovermode='x')

# 📌 8. Interaktives Diagramm für den Niederschlag erstellen
fig_rain = go.Figure()

fig_rain.add_trace(go.Scatter(x=df_grouped['year_group'], y=df_grouped['precipitation_mm'], 
                              mode='lines+markers', name='Durchschnittlicher Niederschlag', 
                              line=dict(color='purple', dash='dash')))

fig_rain.update_layout(title='Niederschlagsentwicklung alle 20 Jahre',
                       xaxis_title='Jahr (20-Jahres-Gruppen)',
                       yaxis_title='Niederschlag (mm)',
                       legend_title='Niederschlagswerte',
                       hovermode='x')

# 📌 9. Diagramme anzeigen
fig_temp.show()
fig_rain.show()


## 10. Auswertung von zusätzlichen Wetterparametern wie Niederschlag und Luftfeuchtigkeit9. 

In [1]:


# 📌 1. Datum in das richtige Format umwandeln
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# 📌 2. Saisonale Klassifikation (Frühling, Sommer, Herbst, Winter)
df['month'] = df['date'].dt.month
df['season'] = np.where(df['month'].isin([12, 1, 2]), 'Winter', 
                        np.where(df['month'].isin([3, 4, 5]), 'Frühling', 
                                 np.where(df['month'].isin([6, 7, 8]), 'Sommer', 
                                          np.where(df['month'].isin([9, 10, 11]), 'Herbst', 'Übergang'))))

# 📌 3. Daten nach Saison gruppieren und Mittelwerte berechnen
df_seasonal = df.groupby('season').agg({
    'avg_temp_c': 'mean',
    'min_temp_c': 'mean',
    'max_temp_c': 'mean',
    'precipitation_mm': 'mean',
  
}).reset_index()

# 📌 4. Temperatur-Visualisierung nach Saison
fig_temp = go.Figure()

fig_temp.add_trace(go.Bar(x=df_seasonal['season'], y=df_seasonal['avg_temp_c'], 
                          name='Durchschnittstemperatur', marker_color='skyblue'))  # لون السماء
fig_temp.add_trace(go.Bar(x=df_seasonal['season'], y=df_seasonal['max_temp_c'], 
                          name='Maximale Temperatur', marker_color='red'))  # لون الأحمر
fig_temp.add_trace(go.Bar(x=df_seasonal['season'], y=df_seasonal['min_temp_c'], 
                          name='Minimale Temperatur', marker_color='green'))  # لون الأخضر

fig_temp.update_layout(title='Saisonale Temperaturunterschiede (Frühling, Sommer, Herbst, Winter)',
                       xaxis_title='Saison',
                       yaxis_title='Temperatur (°C)',
                       legend_title='Temperaturwerte',
                       barmode='group')

# 📌 5. Niederschlags-Visualisierung nach Saison
fig_rain = go.Figure()

fig_rain.add_trace(go.Bar(x=df_seasonal['season'], y=df_seasonal['precipitation_mm'], 
                          name='Durchschnittlicher Niederschlag', marker_color='orange'))  # لون البرتقالي

fig_rain.update_layout(title='Saisonale Niederschlagsunterschiede (Frühling, Sommer, Herbst, Winter)',
                       xaxis_title='Saison',
                       yaxis_title='Niederschlag (mm)',
                       legend_title='Niederschlagswerte')


# 📌 7. Diagramme anzeigen
fig_temp.show()      
fig_rain.show()       
   


NameError: name 'pd' is not defined

## 11. Kombinierte Darstellung mehrerer Wetterparameter in einem Diagramm.

In [ ]:

# 📌 1. Datum in das richtige Format umwandeln
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# 📌 2. Saisonale Klassifikation (Frühling, Sommer, Herbst, Winter)
df['month'] = df['date'].dt.month
df['season'] = np.where(df['month'].isin([12, 1, 2]), 'Winter', 
                        np.where(df['month'].isin([3, 4, 5]), 'Frühling', 
                                 np.where(df['month'].isin([6, 7, 8]), 'Sommer', 
                                          np.where(df['month'].isin([9, 10, 11]), 'Herbst', 'Übergang'))))

# 📌 3. Daten nach Saison gruppieren und Mittelwerte berechnen
df_seasonal = df.groupby('season').agg({
    'avg_temp_c': 'mean',
    'min_temp_c': 'mean',
    'max_temp_c': 'mean',
    'precipitation_mm': 'mean',
    'humidity': 'mean'  #
}).reset_index()

# 📌 4. Kombinierte Visualisierung der Wetterparameter (Temperatur, Niederschlag, Luftfeuchtigkeit)
fig_combined = go.Figure()

fig_combined.add_trace(go.Bar(x=df_seasonal['season'], y=df_seasonal['avg_temp_c'], 
                              name='Durchschnittstemperatur', marker_color='skyblue', yaxis


SyntaxError: incomplete input (2472003619.py, line 29)

## 12. Datenaggregation nach Tag, Woche, Monat

In [ ]:
# Datenaggregation nach Tag, Woche, Monat
df['week'] = pd.to_datetime(df['date']).dt.isocalendar().week
df_weekly = df.groupby('week')[['avg_temp_c', 'precipitation_mm']].mean()
print("Wöchentliche Durchschnittswerte:")
print(df_weekly.head())
#....

Wöchentliche Durchschnittswerte:
      avg_temp_c  precipitation_mm
week                              
1      49.035630          3.085179
2      48.787663          2.998007
3      48.644711          2.972176
4      48.848529          3.001164
5      49.232630          3.001550


## 13. Zusammenfassung der wichtigsten Wetterkenn

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# 🛠️ Initialisierung der Dash-Anwendung
app = dash.Dash(__name__)

# 📌 1. Vorbereiten der Daten
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['month'] = df['date'].dt.month
df['season'] = np.where(df['month'].isin([12, 1, 2]), 'Winter', 
                        np.where(df['month'].isin([3, 4, 5]), 'Frühling', 
                                 np.where(df['month'].isin([6, 7, 8]), 'Sommer', 
                                          np.where(df['month'].isin([9, 10, 11]), 'Herbst', 'Übergang'))))

# Berechnung der Saison-Durchschnittswerte und -Minima/Maxima
summary = df.groupby('season').agg({
    'avg_temp_c': 'mean',
    'min_temp_c': 'min',
    'max_temp_c': 'max',
    'precipitation_mm': 'sum',
}).reset_index()

# 📌 2. Erstellen der Diagramme
# Temperatur-Diagramm
temp_fig = go.Figure()
temp_fig.add_trace(go.Bar(x=summary['season'], y=summary['avg_temp_c'], name='Durchschnittstemperatur', marker_color='skyblue'))
temp_fig.add_trace(go.Bar(x=summary['season'], y=summary['max_temp_c'], name='Maximale Temperatur', marker_color='red'))
temp_fig.add_trace(go.Bar(x=summary['season'], y=summary['min_temp_c'], name='Minimale Temperatur', marker_color='green'))

temp_fig.update_layout(
    title='Durchschnittliche Temperaturen je Saison',
    xaxis_title='Saison',
    yaxis_title='Temperatur (°C)',
    barmode='group'  # Setzt die Balkendiagramme nebeneinander
)

# Niederschlags-Diagramm
rain_fig = go.Figure()
rain_fig.add_trace(go.Bar(x=summary['season'], y=summary['precipitation_mm'], name='Niederschlag (mm)', marker_color='orange'))

rain_fig.update_layout(
    title='Gesamt Niederschlag je Saison',
    xaxis_title='Saison',
    yaxis_title='Niederschlag (mm)',
)

# 📌 3. Aufbau des Dashboards (Übersichtliche Darstellung)
app.layout = html.Div([
    html.H1("Wetterdashboard: Saisonale Übersicht", style={'text-align': 'center'}),  # Überschrift
    html.Div([
        # Zwei Diagramme nebeneinander für Temperatur und Niederschlag
        html.Div([
            dcc.Graph(figure=temp_fig, style={'width': '48%', 'display': 'inline-block'}),
            dcc.Graph(figure=rain_fig, style={'width': '48%', 'display': 'inline-block'}),
        ], style={'display': 'flex', 'justify-content': 'space-between'}),  # Flexbox für Layout
    ], style={'width': '80%', 'margin': '0 auto'})  # Zentrale Ausrichtung des Dashboards
])

# 🛠️ Starten der Dash-Anwendung
if __name__ == '__main__':
    app.run_server(debug=True)
